In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [2]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_yvle_mingxing=pd.DataFrame(columns=columns)
channel_yvle_mingxing

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [13]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [14]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[13]/div/a/span').click()  #选择“娱乐”频道

In [6]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[13]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'娱乐'

In [7]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[3]/a').click()  #选择“明星”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[3]/a').text
sub_channel

'明星'

In [8]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [21]:
print(datetime.datetime.now())

for month in range(6,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_yvle_mingxing.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-23 14:28:58.183779


<ipython-input-21-e6ed4623d94d>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-21-e6ed4623d94d>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-21-e6ed4623d94d>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1Ri4y1s7Xq视频属性的抓取


E:\anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


BV号为1Ri4y1s7Xq的视频属性抓取结束


...

开始BV号为16t4y197EF视频属性的抓取
BV号为16t4y197EF的视频属性抓取结束
开始BV号为1Fa4y1e7ZN视频属性的抓取
BV号为1Fa4y1e7ZN的视频属性抓取结束
开始BV号为1bz411i7ja视频属性的抓取
BV号为1bz411i7ja的视频属性抓取结束
开始BV号为1Fg4y1q73R视频属性的抓取
BV号为1Fg4y1q73R的视频属性抓取结束
开始BV号为1QZ4y1H7xN视频属性的抓取
BV号为1QZ4y1H7xN的视频属性抓取结束
开始BV号为1kk4y1q74Y视频属性的抓取
BV号为1kk4y1q74Y的视频属性抓取结束
开始BV号为18K411p7aR视频属性的抓取
BV号为18K411p7aR的视频属性抓取结束
开始BV号为15A411i7p6视频属性的抓取
BV号为15A411i7p6的视频属性抓取结束
开始BV号为15V411k7Dh视频属性的抓取
BV号为15V411k7Dh的视频属性抓取结束
开始BV号为1Ai4y1V7ob视频属性的抓取
BV号为1Ai4y1V7ob的视频属性抓取结束
开始BV号为1NK4y147rm视频属性的抓取
BV号为1NK4y147rm的视频属性抓取结束
开始BV号为1JD4y1Q7nQ视频属性的抓取
BV号为1JD4y1Q7nQ的视频属性抓取结束
开始BV号为1ct4y197sJ视频属性的抓取
BV号为1ct4y197sJ的视频属性抓取结束
开始BV号为1ha4y1v744视频属性的抓取
BV号为1ha4y1v744的视频属性抓取结束
开始BV号为1K5411Y7K4视频属性的抓取
BV号为1K5411Y7K4的视频属性抓取结束
开始BV号为1cf4y117yJ视频属性的抓取
BV号为1cf4y117yJ的视频属性抓取结束
开始BV号为12A411v7hZ视频属性的抓取
BV号为12A411v7hZ的视频属性抓取结束
开始BV号为1gt4y1y7ro视频属性的抓取
BV号为1gt4y1y7ro的视频属性抓取结束
开始BV号为1FZ4y1p7SX视频属性的抓取
BV号为1FZ4y1p7SX的视频属性抓取结束
开始BV号为1wp4y1D7Yn视频属性的抓取
BV号为1wp4y1D7Yn的视频属性抓取结束
开始BV号为1Bz4y1Q7qU视频属性的抓取
BV号为1Bz4y1Q7qU的视

<ipython-input-21-e6ed4623d94d>:187: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为18k4y1z7Mv视频属性的抓取
BV号为18k4y1z7Mv的视频属性抓取结束
开始BV号为1wi4y1x75F视频属性的抓取
BV号为1wi4y1x75F的视频属性抓取结束
开始BV号为1hp4y1D7XB视频属性的抓取
BV号为1hp4y1D7XB的视频属性抓取结束
开始BV号为1eD4y1D7Aq视频属性的抓取
BV号为1eD4y1D7Aq的视频属性抓取结束
开始BV号为1MV411k7yR视频属性的抓取
BV号为1MV411k7yR的视频属性抓取结束
开始BV号为1ez411i7S8视频属性的抓取
BV号为1ez411i7S8的视频属性抓取结束
开始BV号为1ei4y1x7VH视频属性的抓取
BV号为1ei4y1x7VH的视频属性抓取结束
开始BV号为1nD4y1Q79f视频属性的抓取
BV号为1nD4y1Q79f的视频属性抓取结束
开始BV号为1BV411r76U视频属性的抓取
BV号为1BV411r76U的视频属性抓取结束
开始BV号为1di4y1G7VH视频属性的抓取
BV号为1di4y1G7VH的视频属性抓取结束
开始BV号为1iQ4y1P7GR视频属性的抓取
BV号为1iQ4y1P7GR的视频属性抓取结束
开始BV号为1mC4y1a7k1视频属性的抓取
BV号为1mC4y1a7k1的视频属性抓取结束
开始BV号为1dC4y1a7V3视频属性的抓取
BV号为1dC4y1a7V3的视频属性抓取结束
开始BV号为11C4y1a7Vy视频属性的抓取
BV号为11C4y1a7Vy的视频属性抓取结束
开始BV号为1sk4y1q7CV视频属性的抓取
BV号为1sk4y1q7CV的视频属性抓取结束
开始BV号为17C4y1b7dy视频属性的抓取
BV号为17C4y1b7dy的视频属性抓取结束
开始BV号为1ng4y1i7Ke视频属性的抓取
BV号为1ng4y1i7Ke的视频属性抓取结束
开始BV号为1Bi4y1371Y视频属性的抓取
BV号为1Bi4y1371Y的视频属性抓取结束
开始BV号为16p4y1i7K3视频属性的抓取
BV号为16p4y1i7K3的视频属性抓取结束
开始BV号为1HZ4y1u7FV视频属性的抓取
BV号为1HZ4y1u7FV的视频属性抓取结束
开始BV号为1m5411Y7g7视频属性的抓取
BV号为1m5411Y7g7的视

开始BV号为1B54y1U7JE视频属性的抓取
BV号为1B54y1U7JE的视频属性抓取结束
开始BV号为1oK411J7cg视频属性的抓取
BV号为1oK411J7cg的视频属性抓取结束
开始BV号为1Dz4y1D7EB视频属性的抓取
BV号为1Dz4y1D7EB的视频属性抓取结束
开始BV号为1pD4y127Tk视频属性的抓取
BV号为1pD4y127Tk的视频属性抓取结束
开始BV号为1h54y1U7QH视频属性的抓取
BV号为1h54y1U7QH的视频属性抓取结束
开始BV号为1aK411T7zP视频属性的抓取
BV号为1aK411T7zP的视频属性抓取结束
开始BV号为1Sf4y1971F视频属性的抓取
BV号为1Sf4y1971F的视频属性抓取结束
开始BV号为1RD4y1U7QU视频属性的抓取
BV号为1RD4y1U7QU的视频属性抓取结束
开始BV号为1Za4y17715视频属性的抓取
BV号为1Za4y17715的视频属性抓取结束
开始BV号为1hV411U7iB视频属性的抓取
BV号为1hV411U7iB的视频属性抓取结束
开始BV号为1oA411Y7ms视频属性的抓取
BV号为1oA411Y7ms的视频属性抓取结束
开始BV号为1U54y1e7eV视频属性的抓取
BV号为1U54y1e7eV的视频属性抓取结束
开始BV号为1uf4y1d72z视频属性的抓取
BV号为1uf4y1d72z的视频属性抓取结束
开始BV号为1kk4y127vM视频属性的抓取
BV号为1kk4y127vM的视频属性抓取结束
开始BV号为1WK411T7oR视频属性的抓取
BV号为1WK411T7oR的视频属性抓取结束
开始BV号为1et4y1D714视频属性的抓取
BV号为1et4y1D714的视频属性抓取结束
开始BV号为1Vi4y1g7qT视频属性的抓取
BV号为1Vi4y1g7qT的视频属性抓取结束
开始BV号为1Ya4y1J7ka视频属性的抓取
BV号为1Ya4y1J7ka的视频属性抓取结束
开始BV号为1dV411z7Hs视频属性的抓取
BV号为1dV411z7Hs的视频属性抓取结束
开始BV号为1Zy4y1C7Qr视频属性的抓取
BV号为1Zy4y1C7Qr的视频属性抓取结束
开始BV号为1dh411d7Gm视频属性的抓取
BV号为1dh411d7Gm的视

BV号为1mA411L7Yr的视频属性抓取结束
开始BV号为1jr4y1w7BK视频属性的抓取
BV号为1jr4y1w7BK的视频属性抓取结束
开始BV号为1Vt4y1i7Us视频属性的抓取
BV号为1Vt4y1i7Us的视频属性抓取结束
开始BV号为1aA41177Ev视频属性的抓取
BV号为1aA41177Ev的视频属性抓取结束
开始BV号为1Sa411A7ur视频属性的抓取
BV号为1Sa411A7ur的视频属性抓取结束
开始BV号为1r54y1179E视频属性的抓取
BV号为1r54y1179E的视频属性抓取结束
开始BV号为1Z54y1674u视频属性的抓取
BV号为1Z54y1674u的视频属性抓取结束
开始BV号为16k4y1C7nc视频属性的抓取
BV号为16k4y1C7nc的视频属性抓取结束
开始BV号为1Di4y1j7bQ视频属性的抓取
BV号为1Di4y1j7bQ的视频属性抓取结束
开始BV号为1UK4y1h7YG视频属性的抓取
BV号为1UK4y1h7YG的视频属性抓取结束
开始BV号为1da411w792视频属性的抓取
BV号为1da411w792的视频属性抓取结束
开始BV号为1tf4y1i7wH视频属性的抓取
BV号为1tf4y1i7wH的视频属性抓取结束
开始BV号为13K4y1h7GD视频属性的抓取
BV号为13K4y1h7GD的视频属性抓取结束
开始BV号为1tZ4y1V7Zz视频属性的抓取
BV号为1tZ4y1V7Zz的视频属性抓取结束
开始BV号为19Z4y1G7Qb视频属性的抓取
BV号为19Z4y1G7Qb的视频属性抓取结束
开始BV号为1Rp4y1r7b6视频属性的抓取
BV号为1Rp4y1r7b6的视频属性抓取结束
开始BV号为1Zz4y1Z7sW视频属性的抓取
BV号为1Zz4y1Z7sW的视频属性抓取结束
开始BV号为1mK4y1h7jq视频属性的抓取
BV号为1mK4y1h7jq的视频属性抓取结束
开始BV号为1My4y1r7ty视频属性的抓取
BV号为1My4y1r7ty的视频属性抓取结束
开始BV号为1zA411j7qH视频属性的抓取
BV号为1zA411j7qH的视频属性抓取结束
开始BV号为1S54y117Zu视频属性的抓取
BV号为1S54y117Zu的视频属性抓取结束
开始BV号为18f4y1B7RN

In [20]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-20-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [22]:
channel_yvle_mingxing.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
7月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
8月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [23]:
channel_yvle_mingxing.to_json('channel_yvle_mingxing.json')

In [24]:
channel_yvle_mingxing

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
1HJ411n7Eo,1月,2020-01-12 22:57:26,一只南音呀,个人认证,69.7万,494,【SBS综艺】Running Man 2020合集开始（更新E531.201129 顶楼竞赛...,娱乐,明星,普通视频,...,"[梁世灿, 全昭旻, 池石镇, 宋智孝, 金钟国, 刘在石, Running Man, 综艺...",84:39,1888.1万,6.4万,3.0万,6.6万,1.3万,4140,47.0万,是
16J411n7PR,1月,2020-01-13 01:54:54,喵儿快回家,无,59.6万,1030,【超人回来了】高清全集(更新至E361.201025) KBS综艺超清中字合集持续更新201...,娱乐,明星,普通视频,...,"[建厚, 威廉, 超人回来了, 综艺, Korea相关, 娱乐, 明星, 明星, 本利特, ...",100:06,911.8万,2.6万,3914,3.9万,1.3万,1533,19.4万,是
19J411H71S,1月,2020-01-11 10:31:08,喵儿快回家,无,59.6万,1030,【我独自生活】中字合集(更新至最新E369.201030)Henry刘宪华 MBC综艺完整高...,娱乐,明星,普通视频,...,"[我独自生活, 刘宪华, 华莎, 综艺, Korea相关, 娱乐, 明星, 明星, HENR...",90:08,716.9万,2.3万,3637,4.2万,7195,1830,16.1万,是
1h7411q7Ev,1月,2020-01-19 13:38:33,一只短腿的源柯基,无,1.4万,11,诸葛大力成果·远古直播,娱乐,明星,普通视频,...,"[诸葛大力, 爱情公寓5, 成果, 娱乐, 明星, 明星]",00:30,643.4万,8.9万,1.2万,4.8万,3.8万,3624,9649,否
1xJ411p7b5,1月,2020-01-11 23:24:53,喵儿快回家,无,59.6万,1030,【闲着干嘛呢】高清全集(持更至E65.201024中字)退款远征队 摄像接力综艺 多版本中字...,娱乐,明星,普通视频,...,"[郑允浩, 刘在石, 综艺, Korea相关, 娱乐, 明星, 明星, 刘在锡, 柳熙烈, ...",08:56,581.9万,2.1万,5561,4.1万,8862,1906,21.9万,是
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1Ba411A74v,10月,2020-10-25 00:32:44,LuLu_Uluru,无,1.7万,110,【中字】19年韩国神婆对红毛的Irene的神预测，准了！,娱乐,明星,普通视频,...,"[裴珠泫, REDVELVET, 娱乐, 明星, 明星, 艾琳, KOREA相关, Iren...",00:58,48.4万,2241,57,1262,1620,400,128,否
14K4y1h75M,10月,2020-10-13 18:06:34,犬来八荒w,个人认证,196.6万,11,犬来八荒 | 赛车阿狗！看到最后！,娱乐,明星,普通视频,...,"[成果, 娱乐, 明星, 明星, 狗哥, 犬来八荒, realme, realme真我手机,...",01:25,47.9万,4.6万,1.5万,6056,901,1614,1047,否
1at4y1i7xD,10月,2020-10-26 01:00:06,马思唯,个人认证,34.2万,15,公 开 处 刑,娱乐,明星,活动作品和合作视频,...,"[说唱新世代, 说唱, 娱乐, 明星, 明星, 娱乐, 歌手, 中国boy, 88risin...",03:54,47.7万,1.9万,1433,2285,558,323,1176,否
1gk4y1C7Lb,10月,2020-10-05 18:31:20,忘川不眠秋牧十六,无,475,11,一般来说，能看到该视频的不是绅士就是个爱美人士(？),娱乐,明星,普通视频,...,"[娱乐, 明星, 明星, 女神, 娱乐, 美女, 可爱, 萌妹子]",00:26,47.7万,4228,181,3355,135,206,89,否
